### Import necessary packages

In [2]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt

### Initializing parameters for training the model 

In [3]:
learning_rate = 1e-4
EPOCHS = 20
batch_size = 32

### Directory of dataset and category of data

In [6]:
directory = r"dataset"
categories = ["MASK", "NO_MASK"]

### Load images from the directory and preprocess

In [9]:
data = []
labels = []

for category in categories:
    path = os.path.join(directory, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)
        
        

### one hot encoding for labels

In [11]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

### Converting dada and labels into numpy array

In [12]:
data = np.array(data, dtype="float")
labels = np.array(labels)

### Split data into train and test

In [26]:
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

### Using image data generator to generate more training image

In [15]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

### Load MobileNet model (head fc layers set are left off)

In [16]:
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

### construct the head model and place on the top of the base model

In [17]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

### New Model for our purpose (place head on top of base model)

In [19]:
model = Model(inputs=baseModel.input, outputs=headModel)

### Freez the layers in the base model (should not update in first training process)

In [20]:
for layer in baseModel.layers:
    layer.trainable = False

### Compliling our model

In [24]:
print("[info] compiling model...")

opt = Adam(learning_rate=learning_rate, decay = learning_rate/EPOCHS)

model.compile(loss = "binary_crossentropy", optimizer=opt, metrics=["Accuracy"])

print("[info] compiled successfully")

[info] compiling model...
[info] compiled successfully


### Train the network

In [27]:
print("[info] training head...")

H = model.fit(aug.flow(trainX, trainY, batch_size=batch_size),
             steps_per_epoch=len(trainX)//batch_size,
             validation_data=(testX, testY),
             validation_steps=len(testX)//batch_size,
             epochs=EPOCHS
             )

print("[info] trained successfully")

[info] training head...
Epoch 1/20
48/48 [==============================] - 56s 899ms/step - loss: 0.3455 - accuracy: 0.0000e+00 - val_loss: 0.0775 - val_accuracy: 0.0000e+00
Epoch 2/20
48/48 [==============================] - 27s 547ms/step - loss: 0.1006 - accuracy: 0.0000e+00 - val_loss: 0.0283 - val_accuracy: 0.0000e+00
Epoch 3/20
48/48 [==============================] - 28s 587ms/step - loss: 0.0617 - accuracy: 0.0052 - val_loss: 0.0163 - val_accuracy: 0.0000e+00
Epoch 4/20
48/48 [==============================] - 27s 556ms/step - loss: 0.0422 - accuracy: 0.0187 - val_loss: 0.0111 - val_accuracy: 0.0038
Epoch 5/20
48/48 [==============================] - 26s 550ms/step - loss: 0.0332 - accuracy: 0.0389 - val_loss: 0.0080 - val_accuracy: 0.0179
Epoch 6/20
48/48 [==============================] - 27s 555ms/step - loss: 0.0261 - accuracy: 0.0628 - val_loss: 0.0074 - val_accuracy: 0.0487
Epoch 7/20
48/48 [==============================] - 27s 566ms/step - loss: 0.0272 - accuracy: 0.08

### Prediction on testing set


In [28]:
print("[info] evaluating network...")
predIds = model.predict(testX, batch_size=batch_size)
print("[info] prections ids ready to evalute ...")

[info] evaluating network...
[info] prections ids ready to evalute ...


### Compare True labels with predicted labels (max probability of predicted labels)

In [29]:
predIds = np.argmax(predIds, axis=1)


### Show classification report

In [30]:
print(classification_report(testY.argmax(axis=1), predIds, target_names=lb.classes_))

              precision    recall  f1-score   support

        MASK       0.99      1.00      1.00       190
     NO_MASK       1.00      0.99      1.00       200

    accuracy                           1.00       390
   macro avg       1.00      1.00      1.00       390
weighted avg       1.00      1.00      1.00       390



In [ ]:
### save mode